# Test case for WhoIsBlogger (WIB)

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json

from datetime import datetime, timedelta
import random
import psycopg2

## Step 1: initialize parametrs

In [2]:
# Demo data generation params
# set the quantity of lines in each table
Clients_qty = 100
mu, sigma = 30, 20 # Age normal distribution
Items_qty = 100
Orders_qty = 250

In [3]:
# SQL connection (I used Yandex.Cloud PostgreSQL server. You should to correct connection for your case)
# Note bene: I used cert by Yandex for connection. Don't forget to download it, if you need. (https://storage.yandexcloud.net/cloud-certs/CA.pem)
# or remove "ca_cert_path" and "sslmode" keys.
# read credits for db connection
with open('credentials\\credentials.json', 'r') as file:
    connection_params = json.load(file)

## Step 2: generate of demo-data

In [4]:
AgeArray = np.random.normal(mu, sigma, Clients_qty).round(0)
AgeArray = np.where(AgeArray < 18,mu,AgeArray)

user_df = pd.DataFrame({
    'Age':AgeArray,
    'Active':np.random.randint(0,2,Clients_qty)
    }) \
    .reset_index() \
    .rename(columns={'index':'userId'})


items_df = pd.DataFrame({
    'price': np.random.randint(100,10000,Items_qty),
    'Active':np.random.randint(0,2,Clients_qty)
    }) \
    .reset_index() \
    .rename(columns={'index':'itemId'})


In [5]:
# Create sample set of demo-orders
purchases_df = user_df.merge(items_df, on='Active') \
    .sample(Orders_qty) \
    .drop('Active',axis=1) \
    .reset_index() \
    .rename(columns={'index':'purchaseId'})



# Set interval of dates
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)
# Создание списка случайных дат
dates = []
for _ in range(Orders_qty):
    random_date = start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))
    dates.append(random_date)

# add column "date" to df with purchases
purchases_df['date'] = dates

## Step 3: Connection to db and upload data

In [6]:
connection = psycopg2.connect(**connection_params)

q = connection.cursor()
q.execute('SELECT version()')
print(q.fetchone())

('PostgreSQL 15.2 (Ubuntu 15.2-201-yandex.54409.3746b2454e) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',)


In [7]:
# Create Object Engine in SQLAlchemy
engine = create_engine('postgresql+psycopg2://', creator=lambda: connection)

## Step 4: Creation and fill tables

In [8]:
# Export DataFrames to DB
user_df \
    .drop("Active",axis=1) \
    .to_sql('Users', con=engine, if_exists='replace', index=False)
items_df \
    .drop("Active",axis=1) \
    .to_sql('Items', con=engine, if_exists='replace', index=False)

purchases_df \
    .to_sql('Purchases', con=engine, if_exists='replace', index=False)

250

# Tasks
## Task A
А) какую сумму в среднем в месяц тратит:
- пользователи в возрастном диапазоне от 18 до 25 лет включительно
- пользователи в возрастном диапазоне от 26 до 35 лет включительно

In [9]:
q.execute('''
WITH source_table AS (
  SELECT "Items"."price",
    CASE
        WHEN "Users"."Age" BETWEEN 18 AND 25 THEN '18-25 group'
        WHEN "Users"."Age" BETWEEN 26 AND 35 THEN '26-35 group'
    END AS Group_Name
  FROM "Purchases"
  LEFT JOIN "Users" USING("userId")
  LEFT JOIN "Items" USING("itemId")
)

SELECT
    ROUND(AVG("price"), 2) AS Avg_price,
    Group_Name
FROM source_table
WHERE Group_Name IS NOT NULL
GROUP BY Group_Name
''')

column_names = [desc[0] for desc in q.description]
pd.DataFrame(q.fetchall(), columns=column_names)



,avg_price,group_name
0,5344.52,26-35 group
1,5411.91,18-25 group


## Task B
Б) в каком месяце года выручка от пользователей в возрастном диапазоне 35+ самая большая
- формулировку "возрастной диапазон 35+" интерпретирую как >= 35 лет (аналогично распространенной формулировке 18+)

### Case 1
- акцентирую на то что "в каком месяце года" интерпретирую как сравнение всех январей в учётном периоде, всех декабрей и так далее... то есть сравниваем выручку по месяцам усредненную по годам. (Альтернативный подход в Case 2)
Другими словами, мы находим тот месяц, который, в среднем, собирает большую выручку

In [10]:
q.execute('''
WITH source_table AS (SELECT "Items"."price"
                           , date_trunc('month', date) as month_with_year
                           , "Users"."Age"
                      FROM "Purchases"
                               LEFT JOIN "Users" USING ("userId")
                               LEFT JOIN "Items" USING ("itemId")
                      WHERE "Users"."Age" >= 35)


   , stat_by_month_by_year AS (SELECT SUM("price")                            AS revenue
                                    , month_with_year
                                    , trim(to_char(month_with_year, 'Month')) AS month
                               FROM source_table
                               -- WHERE "Age" >= 35
                               GROUP BY month_with_year)

select month
     , round(AVG(revenue), 2) as revenue
from stat_by_month_by_year
GROUP BY month
ORDER BY AVG(revenue) desc
limit 1
''')

# Output print
column_names = [desc[0] for desc in q.description]
pd.DataFrame(q.fetchall(), columns=column_names)

,month,revenue
0,April,38346.50


### Case 2
Нахожу за всё время конкретный месяц, за который была получена наибольшая выручка

In [11]:
q.execute('''
SELECT trim(to_char(date, 'Month'))                                        as month
     , trim(to_char(date, 'YYYY-MM'))                                      as date_month
     , sum(t_older_35.price) OVER (PARTITION BY date_trunc('month', date)) as revenue_by_month
FROM (SELECT date, price
      FROM "Purchases"
               LEFT JOIN "Users" USING ("userId")
      WHERE "Users"."Age" >= 35) t_older_35

ORDER BY revenue_by_month DESC
LIMIT 1
''')

# Output print
column_names = [desc[0] for desc in q.description]
pd.DataFrame(q.fetchall(), columns=column_names)

,month,date_month,revenue_by_month
0,April,2022-04,44925


## Task C
В) какой товар обеспечивает дает наибольший вклад в выручку за последний год
- "за последний год" интерпретирую как за последние 365 дней (альтернативно можно за период с 1 января, такой способ использован в следующем задание)

In [12]:
q.execute('''
SELECT "itemId"
     , sum("Items"."price") as revenue
FROM "Purchases"
         LEFT JOIN "Items" USING ("itemId")
WHERE date >= current_date - interval '365 days'
GROUP BY "itemId"
order by 2 desc
limit 1
''')

# Output print
column_names = [desc[0] for desc in q.description]
pd.DataFrame(q.fetchall(), columns=column_names)

,itemId,revenue
0,79,29000


## Task D
Г) топ-3 товаров по выручке и их доля в общей выручке за любой год
- "любой год" интерпретирую, как год, на моё усмотрение. Буду рассчитать за идущий год


In [13]:
q.execute('''
with revenue_by_item AS (SELECT "itemId"
                              , sum("Items"."price") as revenue
                         FROM "Purchases"
                                  LEFT JOIN "Items" USING ("itemId")
                         WHERE date >= date_trunc('year', current_date)
                         GROUP BY "itemId")


select "itemId"
     , revenue
     , round(revenue * 100.0 / (select sum(revenue) from revenue_by_item), 2) as revenue_share
FROM revenue_by_item
ORDER BY 2 desc
LIMIT 3
''')

# Output print
column_names = [desc[0] for desc in q.description]
pd.DataFrame(q.fetchall(), columns=column_names)

,itemId,revenue,revenue_share
0,4,19590,6.67
1,52,18478,6.29
2,10,17550,5.98
